In [1]:
import base64
import csv
import pandas
from hashlib import md5
from pprint import pprint
import sys
from pathlib import Path

In [2]:
EC = str(Path("~/proj/encoded_client").expanduser())
if EC not in sys.path:
    sys.path.append(EC)
from encoded_client.encoded import ENCODED, DCCValidator

#server = ENCODED("test.encodedcc.org")
server = ENCODED("encd-6248-78c147870-qing.demo.encodedcc.org")
validator = DCCValidator(server)

In [3]:
WRS = str(Path("~/proj/woldlab-rna-seq").expanduser())
if WRS not in sys.path:
    sys.path.append(WRS)
from woldrnaseq import models

In [4]:
dry_run = True

In [5]:
analysis_dir = Path("fullsolo_multi_eoi_container")
log_final = analysis_dir / "Log.final.out"

In [6]:
config = {
    "award": "UM1HG009443",
    "lab": "/labs/barbara-wold/",
    "alignment_step_run":"barbara-wold:starsolo-alignment-step-run",
    "quantification_step_run": "barbara-wold:starsolo-quantification-step-run",
}
bam_accession = "/files/ENCFF315HXX/"
count_accession = ""

In [7]:
df = pandas.read_csv(analysis_dir / "metadata.csv")
df


FileNotFoundError: [Errno 2] No such file or directory: 'fullsolo_multi_eoi_container/metadata.csv'

In [ ]:
df

In [ ]:
!cat $log_final

In [ ]:
star_log = models.load_star_final_log(log_final)

In [ ]:
star_log

In [ ]:
star_log.loc[("", "Mapping speed, Million of reads per hour")]

In [ ]:
# properties from log
star_quality_metric = {
    "Mapping speed, Million of reads per hour": star_log.loc[("", "Mapping speed, Million of reads per hour")],
    "Number of input reads": star_log.loc[("", "Number of input reads")],
    "Average input read length": star_log.loc[("", "Average input read length")],
    
    "Average mapped length": star_log.loc[("UNIQUE READS", "Average mapped length")],
    "Deletion average length": star_log.loc[("UNIQUE READS", "Deletion average length")],
    "Deletion rate per base": "{:0.2f}".format(star_log.loc[("UNIQUE READS", "Deletion rate per base")]),
    "Insertion average length": star_log.loc[("UNIQUE READS", "Insertion average length")],
    "Insertion rate per base": "{:0.2f}".format(star_log.loc[("UNIQUE READS", "Insertion rate per base")]),
    "Mismatch rate per base, %": "{:0.2f}%".format(star_log.loc[("UNIQUE READS", "Mismatch rate per base, %")]),
    "Number of splices: AT/AC": star_log.loc[("UNIQUE READS", "Number of splices: AT/AC")],
    "Number of splices: Annotated (sjdb)": star_log.loc[("UNIQUE READS", "Number of splices: Annotated (sjdb)")],
    "Number of splices: GC/AG": star_log.loc[("UNIQUE READS", "Number of splices: GC/AG")],
    "Number of splices: GT/AG": star_log.loc[("UNIQUE READS", "Number of splices: GT/AG")],
    "Number of splices: Non-canonical": star_log.loc[("UNIQUE READS", "Number of splices: Non-canonical")],
    "Number of splices: Total": star_log.loc[("UNIQUE READS", "Number of splices: Total")],
    "Uniquely mapped reads %": "{:0.2f}".format(star_log.loc[("UNIQUE READS", "Uniquely mapped reads %")]),
    "Uniquely mapped reads number": star_log.loc[("UNIQUE READS", "Uniquely mapped reads number")],

    "Number of reads mapped to multiple loci": star_log.loc[("MULTI-MAPPING READS", "Number of reads mapped to multiple loci")],
    "% of reads mapped to multiple loci": "{:0.2f}%".format(star_log.loc[("MULTI-MAPPING READS", "% of reads mapped to multiple loci")]),
    "Number of reads mapped to too many loci": star_log.loc[("MULTI-MAPPING READS", "Number of reads mapped to too many loci")],
    "% of reads mapped to too many loci": "{:0.2f}%".format(star_log.loc[("MULTI-MAPPING READS", "% of reads mapped to too many loci")]),

    "% of reads unmapped: too many mismatches": "{:0.2f}%".format(star_log.loc[("UNMAPPED READS", "% of reads unmapped: too many mismatches")]),
    "% of reads unmapped: other": "{:0.2f}%".format(star_log.loc[("UNMAPPED READS", "% of reads unmapped: other")]),
    "% of reads unmapped: too short": "{:0.2f}%".format(star_log.loc[("UNMAPPED READS", "% of reads unmapped: too short")]),
    
    "% of chimeric reads": "{:0.2f}%".format(star_log.loc[("CHIMERIC READS", "% of chimeric reads")]),
    "Number of chimeric reads": star_log.loc[("CHIMERIC READS", "Number of chimeric reads")],
    
    # run parameters not from the log file
    #"quality_metric_of": ["/files/TSTFF094770/",],
    "quality_metric_of": [bam_accession],
    "step_run": "barbara-wold:starsolo-alignment-step-run",
    "lab": config["lab"],
    "award": config["award"],
}

# submission properties
#award
#lab
#quality_metric_of
#step_run
#@type?


In [ ]:
validator.validate(star_quality_metric, "star_quality_metric")

In [ ]:
if not dry_run:
    star_quality_metric_uuid = "706fa54a-41d9-4ada-b13a-b3babbadd1dc"
    if star_quality_metric_uuid is None:
        print(server.post_json("/star_quality_metric/", star_quality_metric))
    else:
        pprint(server.get_json(star_quality_metric_uuid))

In [ ]:
# tst_file = server.get_json("/files/TSTFF094770/")
# print(tst_file["quality_metrics"])


In [ ]:
def load_star_solo_quality_metric(analysis_dir, gene_model):
    summary_csv = analysis_dir / "Solo.out"/ gene_model / "Summary.csv"

    # Note need to update to version not using Multipe typo
    summary_dtype = {
        "Number of Reads": int,
        "Reads With Valid Barcodes": float,
        "Sequencing Saturation": float,
        "Q30 Bases in CB+UMI": float,
        "Q30 Bases in RNA read": float,
        "Reads Mapped to Genome: Unique+Multiple": float,
        "Reads Mapped to Genome: Unique": float,
        "Reads Mapped to Gene: Unique+Multipe Gene": float,
        "Reads Mapped to Gene: Unique Gene": float,
        "Reads Mapped to GeneFull: Unique+Multipe GeneFull": float,
        "Reads Mapped to GeneFull: Unique GeneFull": float,
        "Reads Mapped to GeneFull_Ex50pAS: Unique+Multipe GeneFull_Ex50pAS": float,
        "Reads Mapped to GeneFull_Ex50pAS: Unique GeneFull_Ex50pAS": float,
        "Estimated Number of Cells": int,
        "Unique Reads in Cells Mapped to Gene": int,
        "Unique Reads in Cells Mapped to GeneFull": int,
        "Unique Reads in Cells Mapped to GeneFull_Ex50pAS": int,
        "Fraction of Unique Reads in Cells": float,
        "Mean Reads per Cell": int,
        "Median Reads per Cell": int,
        "UMIs in Cells": int,
        "Mean UMI per Cell": int,
        "Median UMI per Cell": int,
        "Mean Gene per Cell": int,
        "Median Gene per Cell": int,
        "Total Gene Detected": int,    
        "Mean GeneFull per Cell": int,
        "Median GeneFull per Cell": int,
        "Total GeneFull Detected": int,    
        "Mean GeneFull_Ex50pAS per Cell": int,
        "Median GeneFull_Ex50pAS per Cell": int,
        "Total GeneFull_Ex50pAS Detected": int,
    }
    summary = {}
    with open(summary_csv, "rt") as instream:
        for name, value in csv.reader(instream):
            summary[name] = summary_dtype[name](value)
    return summary

summary = load_star_solo_quality_metric(analysis_dir, "GeneFull_Ex50pAS")

In [ ]:
def make_attachment(filename, mime_type=None):
    mime_type_default = {
        ".pdf": "application/pdf",
        ".tar": "application/x-tar",
        ".json": "application/json",
        ".gif": "image/gif",
        ".jpg": "image/jpeg",
        ".jpeg": "image/jpeg",
        ".png": "image/png",
        ".tif": "image/tiff",
        ".tiff": "image/tiff",
        ".html": "text/html",
        ".txt": "text/plain",
        ".tsv": "text/tab-separated-values"        
    }

    filename = Path(filename)
    if mime_type is None:
        mime_type = mime_type_default[filename.suffix]
        if mime_type is None:
            raise ValueError("Unrecognized filename extension")

    with open(filename, "rb") as instream:
        document = instream.read()

    short_name = filename.name
    payload = {
        'download': str(short_name),
        'type': mime_type,
        'href': 'data:{};base64,'.format(mime_type) + 
                base64.b64encode(document).decode("ascii"),
        'md5sum': md5(document).hexdigest()
    }
    return payload

In [ ]:
star_solo_metrics = {
    "assay_term_name": "single-cell RNA sequencing assay",
    "barcode_rank_plot": make_attachment(analysis_dir / "umi_per_cell.png"),
    "estimated_number_of_cells": summary["Estimated Number of Cells"],
    "fraction_of_unique_reads_in_cells": summary["Fraction of Unique Reads in Cells"],
    "mean_UMI_per_cell": summary["Mean UMI per Cell"],
    "mean_reads_per_cell": summary["Mean Reads per Cell"],
    "median_UMI_per_cell": summary["Median UMI per Cell"],
    "median_reads_per_cell": summary["Median Reads per Cell"],
    "number_of_reads": summary["Number of Reads"],
    "q30_bases_in_CB_UMI": summary["Q30 Bases in CB+UMI"],
    "q30_bases_in_rna_read": summary["Q30 Bases in RNA read"],
    "reads_mapped_to_genome_unique": summary["Reads Mapped to Genome: Unique"],
    "reads_mapped_to_genome_unique_and_multiple": summary["Reads Mapped to Genome: Unique+Multiple"],
    "reads_with_valid_barcodes": summary["Reads With Valid Barcodes"],
    "sequencing_saturation": summary["Sequencing Saturation"],
    "umis_in_cells": summary["UMIs in Cells"],
    
    # run parameters not from the log file
    "quality_metric_of": [bam_accession,],
    "step_run": config["alignment_step_run"],
    "lab": config["lab"],
    "award": config["award"],   
}

if "Mean Gene per Cell" in summary:
    print("Gene mode")    
    star_solo_metrics.update({
        "mode": "Gene",
        "reads_mapped_to_gene_unique_and_multiple_gene": summary["Reads Mapped to Gene: Unique+Multiple"],
        "reads_mapped_to_gene_unique_gene": summary["Reads Mapped to Gene: Unique"],
        "unique_reads_in_cells_mapped_to_gene": summary["Unique Reads in Cells Mapped to Gene"],
        "mean_gene_per_cell": summary["Mean Gene per Cell"],
        "median_gene_per_cell": summary["Median Gene per Cell"],
        "total_gene_detected": summary["Total Gene Detected"],
    })
elif "Mean GeneFull_Ex50pAS per Cell" in summary:
    print("GeneFull_Ex50pAS mode")
    star_solo_metrics.update({
        "mode": "GeneFull_Ex50pAS",
        "reads_mapped_to_genefull_ex50pas_unique_and_multiple_gene_ex50pas": summary["Reads Mapped to GeneFull_Ex50pAS: Unique+Multipe GeneFull_Ex50pAS"],
        "reads_mapped_to_genefull_ex50pas_unique_genefull_ex50pas": summary["Reads Mapped to GeneFull_Ex50pAS: Unique GeneFull_Ex50pAS"],
        "unique_reads_in_cells_mapped_to_genefull_ex50pas": summary["Unique Reads in Cells Mapped to GeneFull_Ex50pAS"],
        "mean_genefull_ex50pas_per_cell": summary["Mean GeneFull_Ex50pAS per Cell"],
        "median_genefull_ex50pas_per_cell": summary["Median GeneFull_Ex50pAS per Cell"],
        "total_genefull_ex50pas_detected": summary["Total GeneFull_Ex50pAS Detected"],
    })  
elif "Mean GeneFull_Ex50pAS per Cell" in summary:
    print("GeneFull mode")
    star_solo_metrics.update({
        "mode": "GeneFull",
        "reads_mapped_to_genefull_unique_and_multiple_genefull": summary["Reads Mapped to GeneFull: Unique+Multipe GeneFull"],
        "reads_mapped_to_genefull_unique_genefull": summary["Reads Mapped to GeneFull: Unique GeneFull"],
        "unique_reads_in_cells_mapped_to_genefull": summary["Unique Reads in Cells Mapped to GeneFull"],
        "mean_genefull_per_cell": summary["Mean GeneFull_Ex50pAS per Cell"],
        "median_genefull_per_cell": summary["Median GeneFull_Ex50pAS per Cell"],
        "total_genefull_detected": summary["Total GeneFull_Ex50pAS Detected"],
    })
else:
    print("Unknown mode?", sorted(summary.keys()))
#star_solo_metrics

In [ ]:
validator.validate(star_solo_metrics, "star_solo_quality_metric")

In [ ]:
sc_count_metric = {
    "assay_term_name": "single-cell RNA sequencing assay",
    "total_counts_vs_genes_by_count": make_attachment(analysis_dir / "n_genes_by_counts.png"),
    "total_counts_vs_pct_mitochondria": make_attachment(analysis_dir /  "pct_count_mt.png"),
    "qc_metrics_violin": make_attachment(analysis_dir / "qc_metric_violin.png"),
    
    # run parameters not from the log file
    "quality_metric_of": [bam_accession,],
    "step_run": config["quantification_step_run"],
    "lab": config["lab"],
    "award": config["award"],   
}
validator.validate(sc_count_metric, "scrna_seq_counts_summary_quality_metric")

In [ ]:
def generic_quality_metric(config, metric_of, filename, mime_type=None):
    mime_type_default = {
        ".pdf": "application/pdf",
        ".tar": "application/x-tar",
        ".json": "application/json",
        ".gif": "image/gif",
        ".jpg": "image/jpeg",
        ".jpeg": "image/jpeg",
        ".png": "image/png",
        ".tif": "image/tiff",
        ".tiff": "image/tiff",
        ".html": "text/html",
        ".txt": "text/plain",
        ".tsv": "text/tab-separated-values"        
    }

    filename = Path(filename)
    if mime_type is None:
        mime_type = mime_type_default[filename.suffix]
        if mime_type is None:
            raise ValueError("Unrecognized filename extension")

    with open(filename, "rb") as instream:
        document = instream.read()

    short_name = filename.name
    payload = {
        'attachment': {
            'download': str(short_name),
            'type': mime_type,
            'href': 'data:{};base64,'.format(mime_type) + 
                     base64.b64encode(document).decode("ascii"),
            'md5sum': md5(document).hexdigest()
        },
        "name": "UMI detected vs cell barcodes",
        "quality_metric_of": [metric_of],
        'award': config["award"],
        'lab': config["lab"],
        "step_run": config["step_run"],
    }
    return payload

In [ ]:
#generic_metric = generic_quality_metric(config, "/files/TSTFF094770/", analysis_dir / "umi.png")
#generic_metric

In [ ]:
#validator.validate(generic_metric, "generic_quality_metric")

In [ ]:
star_index = server.get_json("ENCFF598IDH")

In [ ]:
sorted(star_index)

In [8]:
from urllib import parse
from encoded_client.encoded import HTTPError
from encoded_client.metadata import format_accession, compute_dcc_file_accession_from_url

In [9]:


def update_genome_annotation_info():
    if "genome_accession" in config:
        genome_accession = config["genome_acession"]
    elif "genome_index_url" in config:
        index_url = config["genome_index_url"]
        genome_accession = compute_dcc_file_accession_from_url(index_url)
    else:
        raise ValueError("genome_accession or genome_index_url are required parameters")
        
    if not('assembly' in config and 'genome_annotation' in config):
        try:
            server = ENCODED(get_submit_host())
            index_metadata = server.get_json(genome_accession)
        except HTTPError as e:
            if e.response.status_code == 404:
                print("{} not found, please set assembly and genone_annotation parameters".format(accession))
            else:
                print("Other error {} for {}".format(e.response.status_code, accession))

        config.update({
            'assembly': index_metadata["assembly"],
            'genome_annotation': index_metadata["genome_annotation"],
        })

index_url = "https://woldlab.caltech.edu/~diane/genome/GRCh38-V29-male-2.7.8a.tar.gz"
print("accession_info", get_accession_info(compute_dcc_file_accession_from_url(index_url)))

index_url = "https://www.encodeproject.org/files/ENCFF598IDH/@@download/ENCFF598IDH.tar.gz"
print("accession_info", get_accession_info(compute_dcc_file_accession_from_url(index_url)))


Error http status: 404 for https://encd-6248-78c147870-qing.demo.encodedcc.org/files/GRCh38-V29-male-2.7.8a/


/files/GRCh38-V29-male-2.7.8a/ not found
accession_info None
accession_info {'assembly': 'GRCh38', 'genome_annotation': 'V29'}


In [12]:
p = parse.urlparse("/files/asdf/")
p.scheme is None or len(p.scheme) == 0

True

In [11]:
p.scheme

''